In [20]:
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(r'C:\Users\RentalHub-L1\Code\NLP\chromedriver.exe')

In [21]:
singer=  ['룰라','쿨','터보','알이에프']
singer_id =[]

In [22]:
for i in singer:
    driver.get("https://www.melon.com/index.htm")   
    search = driver.find_element_by_xpath('//*[@id="top_search"]') #검색창 xpath
    search.clear() #검색창 clear
    search.send_keys(i+'\n')  #검색창에 가수 입력후 엔터
    
    driver.find_element_by_xpath('//*[@id="conts"]/div[3]/div/div[1]/div/a/strong').click() #검색한 아티스트 클릭
    url = driver.current_url
    id = url.replace('https://www.melon.com/artist/timeline.htm?artistId=', '') #url에 있는 가수 고유번호 저장
    
    singer_id.append(id)

In [23]:
print(singer_id)

['106610', '100004', '100059', '100015']


In [24]:
title_ary=[]
date_ary=[]
genre_ary=[]
lyrics_ary=[]
lyricist_ary=[]
composer_ary=[]
singer_ary=[]

In [25]:
def search(id,n):   # 가수id를 받아 (제목,가수,가사, 발매일,장르,작사,작곡)  n개 만큼탐색  
    #id에 맞는 가수의 곡 페이지(인기순)으로 이동
    driver.get('https://www.melon.com/artist/song.htm?artistId='+str(id)+'#params%5BlistType%5D=A&params%5BorderBy%5D=POPULAR_SONG_LIST&params%5BartistId%5D='+str(id)+'&po=pageObj&startIndex=1')
    driver.implicitly_wait(1)

    for i in range(1,n+1):

        driver.execute_script('window.scrollTo(0,200)')
        
        # i번째 곡정보 클릭
        driver.find_element_by_xpath('//*[@id="frm"]/div/table/tbody/tr['+ str(i)+']/td[3]/div/div/a[1]').click()
        
        # 발매일 먼저 검사
        str_tmp=driver.find_element_by_xpath('//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[2]').text
        arr = str_tmp.split('.')
        
        # 발매일이 94년~99년까지인 곡만 정보 수집
        if( int(arr[0]) < 2000 and int(arr[0]) > 1993):
            #print("input: ", str_tmp)
            date_ary.append(str_tmp)
            
            title=driver.find_element_by_xpath('//*[@id="downloadfrm"]/div/div/div[2]/div[1]/div[1]')
            title_ary.append(title.text)
            
            singer=driver.find_element_by_xpath('//*[@id="downloadfrm"]/div/div/div[2]/div[1]/div[2]/a/span[1]')
            singer_ary.append(singer.text)
            
            genre=driver.find_element_by_xpath('//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[3]')
            genre_ary.append(genre.text)
            
            try:
                lyricist=driver.find_element_by_xpath('//*[@id="conts"]/div[3]/ul/li[1]/div[2]/div[1]/a')
                lyricist_ary.append(lyricist.text)
            except:
                lyricist_ary.append('NULL')
        
            try:
                composer=driver.find_element_by_xpath('//*[@id="conts"]/div[3]/ul/li[2]/div[2]/div[1]/a')
                composer_ary.append(composer.text)
            except:
                composer_ary.append('NULL')
        
            try:
                lyrics=driver.find_element_by_xpath('//*[@id="d_video_summary"]')
                lyrics_ary.append(lyrics.text.replace('\n',' '))
            except:
                lyrics_ary.append('NULL')
            
            driver.back() #뒤로가기 ->다시 곡 목록으로 이동
            
        else:
            driver.back()

In [26]:
for i in singer_id:
    search(i,20)

In [27]:
import pandas as pd
df=pd.DataFrame({"년도":date_ary,"가수":singer_ary,"제목":title_ary,"장르":genre_ary,
                 "작사가":composer_ary,"작곡가":lyricist_ary,"가사":lyrics_ary})

In [28]:
df

,년도,가수,제목,장르,작사가,작곡가,가사
0,1997.04.01,룰라,3!4!,댄스,이현도,이현도,여기 숨쉬는 이 시간은 나를 어데로 데려갈까 많은 기쁨과 한숨들이 뒤섞인 이곳에서 ...
1,1995,룰라,날개 잃은 천사,댄스,최준영,이건우,아 그럴거야 나를 아낄려고 굳이 내게 말 안하고 멀리 떠나갔던가 아 그럴거야 나를 ...
2,1996.06.15,룰라,3!4!,댄스,이현도,이현도,여기 숨쉬는 이 시간은 나를 어데로 데려갈까 많은 기쁨과 한숨들이 뒤섞인 이 곳에서...
3,1995,룰라,천상유애(天上有愛),댄스,The Season,이건우,저 끝에서 끝까지 목이 메이도록 말하고 말거야 무척 소중한걸 안다고 거리마다 외칠 ...
4,1997,룰라,연인,댄스,이상민,이상민,바보 바보 바보야 바보 바보 바보야 바보 바보 바보야 사랑 앞에서 오늘도 넌 튕겨 ...
5,1995,룰라,프로와 아마츄어,댄스,최준영,룰라,나는 나는 모든걸 할수가 있어 너가 나를 사랑하게 할수도 있고 너를 감동 시킬수도 ...
6,1997.04.01,룰라,비밀은 없어,댄스,박선만,박선만,우우 정말이야 이제 그대에게 비밀은 없어 우우 알고 있지 사랑한단 말은 안해도 처음...
7,1995,룰라,사랑법,댄스,The Season,룰라,하지만 사랑이 자리잡혀 간다면 진실한 사랑이 필요하기 시작한거야 갖기 위해서 해왔던...
8,1999.02,룰라,기도 (Good),댄스,최준영,이상민,Roo'Ra deeper lover live lover live 라라따리 라리라 라...
9,1999.02,룰라,Moving,댄스,박근태,백종열,가나봐 내가 떠나가도 내가 가나봐 그래봐라 네가 떠나가도 아무렇지 않다고 오지마 너...


In [29]:
idol_list = pd.read_csv('94to95_idol_list.csv')
idol_list.shape

(4, 7)

In [30]:
idol_list.columns

Index(['artist', 'artist_eng', 'member', 'leader', 'gender', 'agency',
       'idol_id'],
      dtype='object')

In [31]:
idol_list.columnsadd_gender = []
add_gender = idol_list.gender
add_gender

C:\Users\RentalHub-L1\anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


0    혼
1    혼
2    남
3    남
Name: gender, dtype: object

In [32]:
# 성별 영어로 바꿔주기

for i in range(len(add_gender)):
    if(add_gender[i] == '남'):
        add_gender[i] = 'male'
    else:
        add_gender[i] = 'mixed'

add_gender

C:\Users\RentalHub-L1\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\RentalHub-L1\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


0    mixed
1    mixed
2     male
3     male
Name: gender, dtype: object

In [34]:
add_agency = []
add_agency = idol_list.agency
add_agency

0    서울인 프로덕션
1        쿨컴퍼니
2    더 터보 컴퍼니
3    팀 엔터테인먼트
Name: agency, dtype: object

In [35]:
test_list = [title_ary, date_ary, genre_ary, lyricist_ary, lyrics_ary]
column_names = ['년도','가수','제목','장르','작사가', '가사']
column_v = [date_ary, singer_ary, title_ary, genre_ary, lyricist_ary, lyrics_ary]

test_df = pd.DataFrame(column_v)

In [36]:
test_p = test_df.head(6).T
test_p.head()

,0,1,2,3,4,5
0,1997.04.01,룰라,3!4!,댄스,이현도,여기 숨쉬는 이 시간은 나를 어데로 데려갈까 많은 기쁨과 한숨들이 뒤섞인 이곳에서 ...
1,1995,룰라,날개 잃은 천사,댄스,이건우,아 그럴거야 나를 아낄려고 굳이 내게 말 안하고 멀리 떠나갔던가 아 그럴거야 나를 ...
2,1996.06.15,룰라,3!4!,댄스,이현도,여기 숨쉬는 이 시간은 나를 어데로 데려갈까 많은 기쁨과 한숨들이 뒤섞인 이 곳에서...
3,1995,룰라,천상유애(天上有愛),댄스,이건우,저 끝에서 끝까지 목이 메이도록 말하고 말거야 무척 소중한걸 안다고 거리마다 외칠 ...
4,1997,룰라,연인,댄스,이상민,바보 바보 바보야 바보 바보 바보야 바보 바보 바보야 사랑 앞에서 오늘도 넌 튕겨 ...


In [58]:
c_artist = []
c_artist = test_p[1]

In [59]:
c_artist

0           룰라
1           룰라
2           룰라
3           룰라
4           룰라
5           룰라
6           룰라
7           룰라
8           룰라
9           룰라
10          룰라
11          룰라
12          룰라
13          룰라
14          룰라
15    쿨 (COOL)
16    쿨 (COOL)
17    쿨 (COOL)
18    쿨 (COOL)
19    쿨 (COOL)
20    쿨 (COOL)
21    쿨 (COOL)
22          터보
23          터보
24          터보
25          터보
26          터보
27          터보
28          터보
29          터보
30          터보
31          터보
32          터보
33          터보
34          터보
35          터보
36        R.ef
37        R.ef
38        R.ef
39        R.ef
40        R.ef
41        R.ef
42        R.ef
43        R.ef
44        R.ef
45        R.ef
46        R.ef
47        R.ef
48        R.ef
49        R.ef
50        R.ef
51        R.ef
52        R.ef
53        R.ef
Name: 1, dtype: object

In [60]:
a_gender = []

for i in range(len(c_artist)) :
    if( c_artist[i] == "룰라"):
        a_gender.append(add_gender[0])
    elif( c_artist[i] == "쿨 (COOL)" ):
        a_gender.append(add_gender[1])
    elif( c_artist[i] == "터보"):
        a_gender.append(add_gender[2])
    else :
        a_gender.append(add_gender[3])
#a_gender

In [61]:
a_gender

['mixed',
 'mixed',
 'mixed',
 'mixed',
 'mixed',
 'mixed',
 'mixed',
 'mixed',
 'mixed',
 'mixed',
 'mixed',
 'mixed',
 'mixed',
 'mixed',
 'mixed',
 'mixed',
 'mixed',
 'mixed',
 'mixed',
 'mixed',
 'mixed',
 'mixed',
 'male',
 'male',
 'male',
 'male',
 'male',
 'male',
 'male',
 'male',
 'male',
 'male',
 'male',
 'male',
 'male',
 'male',
 'male',
 'male',
 'male',
 'male',
 'male',
 'male',
 'male',
 'male',
 'male',
 'male',
 'male',
 'male',
 'male',
 'male',
 'male',
 'male',
 'male',
 'male']

In [62]:
a_agency = []

for i in range(len(c_artist)) :
    if( c_artist[i] == "룰라"):
        a_agency.append(add_agency[0])
    elif( c_artist[i] =="쿨 (COOL)"):
        a_agency.append(add_agency[1])
    elif( c_artist[i] == "터보"):
        a_agency.append(add_agency[2])
    else:
        a_agency.append(add_agency[3])

#a_agency

In [63]:
a_agency

['서울인 프로덕션',
 '서울인 프로덕션',
 '서울인 프로덕션',
 '서울인 프로덕션',
 '서울인 프로덕션',
 '서울인 프로덕션',
 '서울인 프로덕션',
 '서울인 프로덕션',
 '서울인 프로덕션',
 '서울인 프로덕션',
 '서울인 프로덕션',
 '서울인 프로덕션',
 '서울인 프로덕션',
 '서울인 프로덕션',
 '서울인 프로덕션',
 '쿨컴퍼니',
 '쿨컴퍼니',
 '쿨컴퍼니',
 '쿨컴퍼니',
 '쿨컴퍼니',
 '쿨컴퍼니',
 '쿨컴퍼니',
 '더 터보 컴퍼니',
 '더 터보 컴퍼니',
 '더 터보 컴퍼니',
 '더 터보 컴퍼니',
 '더 터보 컴퍼니',
 '더 터보 컴퍼니',
 '더 터보 컴퍼니',
 '더 터보 컴퍼니',
 '더 터보 컴퍼니',
 '더 터보 컴퍼니',
 '더 터보 컴퍼니',
 '더 터보 컴퍼니',
 '더 터보 컴퍼니',
 '더 터보 컴퍼니',
 '팀 엔터테인먼트',
 '팀 엔터테인먼트',
 '팀 엔터테인먼트',
 '팀 엔터테인먼트',
 '팀 엔터테인먼트',
 '팀 엔터테인먼트',
 '팀 엔터테인먼트',
 '팀 엔터테인먼트',
 '팀 엔터테인먼트',
 '팀 엔터테인먼트',
 '팀 엔터테인먼트',
 '팀 엔터테인먼트',
 '팀 엔터테인먼트',
 '팀 엔터테인먼트',
 '팀 엔터테인먼트',
 '팀 엔터테인먼트',
 '팀 엔터테인먼트',
 '팀 엔터테인먼트']

In [64]:
rank = []

In [65]:
column_names = ['년도','가수','제목','남녀','장르','최고순위','작사가','작곡가','소속사' ,'가사']
column_v = [date_ary, singer_ary, title_ary, a_gender, genre_ary,rank,lyricist_ary,composer_ary,a_agency, lyrics_ary]
test_df = pd.DataFrame(column_v)

In [66]:
test_p = test_df.head(10).T
test_p.head()

,0,1,2,3,4,5,6,7,8,9
0,1997.04.01,룰라,3!4!,mixed,댄스,None,이현도,이현도,서울인 프로덕션,여기 숨쉬는 이 시간은 나를 어데로 데려갈까 많은 기쁨과 한숨들이 뒤섞인 이곳에서 ...
1,1995,룰라,날개 잃은 천사,mixed,댄스,None,이건우,최준영,서울인 프로덕션,아 그럴거야 나를 아낄려고 굳이 내게 말 안하고 멀리 떠나갔던가 아 그럴거야 나를 ...
2,1996.06.15,룰라,3!4!,mixed,댄스,None,이현도,이현도,서울인 프로덕션,여기 숨쉬는 이 시간은 나를 어데로 데려갈까 많은 기쁨과 한숨들이 뒤섞인 이 곳에서...
3,1995,룰라,천상유애(天上有愛),mixed,댄스,None,이건우,The Season,서울인 프로덕션,저 끝에서 끝까지 목이 메이도록 말하고 말거야 무척 소중한걸 안다고 거리마다 외칠 ...
4,1997,룰라,연인,mixed,댄스,None,이상민,이상민,서울인 프로덕션,바보 바보 바보야 바보 바보 바보야 바보 바보 바보야 사랑 앞에서 오늘도 넌 튕겨 ...


In [67]:
test_p.columns = column_names
test_p.tail(350)

,년도,가수,제목,남녀,장르,최고순위,작사가,작곡가,소속사,가사
0,1997.04.01,룰라,3!4!,mixed,댄스,None,이현도,이현도,서울인 프로덕션,여기 숨쉬는 이 시간은 나를 어데로 데려갈까 많은 기쁨과 한숨들이 뒤섞인 이곳에서 ...
1,1995,룰라,날개 잃은 천사,mixed,댄스,None,이건우,최준영,서울인 프로덕션,아 그럴거야 나를 아낄려고 굳이 내게 말 안하고 멀리 떠나갔던가 아 그럴거야 나를 ...
2,1996.06.15,룰라,3!4!,mixed,댄스,None,이현도,이현도,서울인 프로덕션,여기 숨쉬는 이 시간은 나를 어데로 데려갈까 많은 기쁨과 한숨들이 뒤섞인 이 곳에서...
3,1995,룰라,천상유애(天上有愛),mixed,댄스,None,이건우,The Season,서울인 프로덕션,저 끝에서 끝까지 목이 메이도록 말하고 말거야 무척 소중한걸 안다고 거리마다 외칠 ...
4,1997,룰라,연인,mixed,댄스,None,이상민,이상민,서울인 프로덕션,바보 바보 바보야 바보 바보 바보야 바보 바보 바보야 사랑 앞에서 오늘도 넌 튕겨 ...
5,1995,룰라,프로와 아마츄어,mixed,댄스,None,룰라,최준영,서울인 프로덕션,나는 나는 모든걸 할수가 있어 너가 나를 사랑하게 할수도 있고 너를 감동 시킬수도 ...
6,1997.04.01,룰라,비밀은 없어,mixed,댄스,None,박선만,박선만,서울인 프로덕션,우우 정말이야 이제 그대에게 비밀은 없어 우우 알고 있지 사랑한단 말은 안해도 처음...
7,1995,룰라,사랑법,mixed,댄스,None,룰라,The Season,서울인 프로덕션,하지만 사랑이 자리잡혀 간다면 진실한 사랑이 필요하기 시작한거야 갖기 위해서 해왔던...
8,1999.02,룰라,기도 (Good),mixed,댄스,None,이상민,최준영,서울인 프로덕션,Roo'Ra deeper lover live lover live 라라따리 라리라 라...
9,1999.02,룰라,Moving,mixed,댄스,None,백종열,박근태,서울인 프로덕션,가나봐 내가 떠나가도 내가 가나봐 그래봐라 네가 떠나가도 아무렇지 않다고 오지마 너...


In [68]:
test_p.shape

(54, 10)

In [69]:
# 곡 전처리
test_p['제목'].isnull().sum()

0

In [70]:
test_p.tail()

,년도,가수,제목,남녀,장르,최고순위,작사가,작곡가,소속사,가사
49,1999,R.ef,상심(Ballad Ver.),male,발라드,None,성대현,이성욱,팀 엔터테인먼트,사랑했던 나의 마음속에 작은 꿈 하나만을 남겨두고 너무 쉽게 나를 떠나버린 너를 이...
50,1996.04,R.ef,니가 뭔데,male,댄스,None,강은경,이용민,팀 엔터테인먼트,시간 남아 사랑하진 안찮아 하루가 모자라게 바쁜 사람도 사랑할 시간만은 아껴두던데 ...
51,1997,R.ef,후애,male,댄스,None,하해룡,이용민,팀 엔터테인먼트,얼마나 소중했는지 그때는 몰랐던거야 지난날 너의 사랑이 내게 남아 상처가 될줄 모랐...
52,1996.09.01,R.ef,마음속을 걸어가 (Jungle Ver.),male,댄스,None,홍재선,홍재선,팀 엔터테인먼트,넌 지금 어딨는지 난 중요치 않아 누구와 함께 있는것이 중요해 나 아닌 다른 사람을...
53,1998.05.01,R.ef,하늘을 걸고...,male,댄스,None,강은경,Papertonic,팀 엔터테인먼트,Hey now come on to me baby Why can't you hold ...


In [71]:
test_p.to_csv('kpop_debut_9495.tsv',index=False,header=None,sep='\t')